In [1]:
import json

In [2]:
with open('N06-19_HighwaySection.geojson') as f:
    highways = json.load(f)['features']

with open('N06-19_Joint.geojson') as f:
    joints = json.load(f)['features']

In [3]:
def tuplize_coordinate(coordinate):
    return (coordinate[0], coordinate[1])

In [4]:
ic_and_jct = {}
for j in joints:
    ic_and_jct[tuplize_coordinate(j['geometry']['coordinates'])] = j['properties']['地点名']

In [40]:
"""
graphのキーはIC・JCT名，
valueは隣接してゐるIC・JCTのリストで，その要素は
{
    name: IC・JCT名,
    road: 路線名
}
"""
graph = {}

# graphの初期化
for ic in ic_and_jct.values():    
    graph[ic] = []

for h in highways:
    points = [ic_and_jct[tuplize_coordinate(c)] for c in h['geometry']['coordinates'] if tuplize_coordinate(c) in ic_and_jct]

    if len(points) <= 1:
        continue

    name = h['properties']['路線名']

    for i in range(len(points)-1):
        ic1 = points[i]
        ic2 = points[i+1]
        graph[ic1].append({'name': ic2, 'road': name})
        graph[ic2].append({'name': ic1, 'road': name})
    

In [41]:
# 重複を削除
for k, v in graph.items():
    tmp = []
    names = []
    for node in v:
        if node['name'] not in names:
            names.append(node['name'])
            tmp.append(node)
        
    graph[k] = tmp

with open('highways_data2.json', 'w') as f:
    json.dump(graph, f, indent=2, ensure_ascii=False)